In [1]:
import requests
import pyarrow as pa
import pyarrow.fs
import io

In [2]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 7696498688 (7.17 GB)
DFS Remaining: 7438815232 (6.93 GB)
DFS Used: 257683456 (245.75 MB)
DFS Used%: 3.35%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 91
	Missing blocks (with replication factor 1): 91
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.18.0.5:9866 (project-4-p4_yw_ym-dn-2.project-4-p4_yw_ym_default)
Hostname: 43dc2ad30e77
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 257683456 (245.75 MB)
Non DFS Used: 18107777024 (16.86 GB)
DFS Remaining: 7438815232 (6.93 GB)
DFS Used%: 1.00%
DFS Remai

In [3]:
url = "http://boss:9870/webhdfs/v1/single.csv?op=LISTSTATUS"
response = requests.get(url)
response.raise_for_status()
file = response.json()
file_dict = file["FileStatuses"]
file_dict

{'FileStatus': [{'accessTime': 1710461499049,
   'blockSize': 1048576,
   'childrenNum': 0,
   'fileId': 16386,
   'group': 'supergroup',
   'length': 174944099,
   'modificationTime': 1710457772795,
   'owner': 'root',
   'pathSuffix': '',
   'permission': '644',
   'replication': 1,
   'storagePolicy': 0,
   'type': 'FILE'}]}

In [4]:
size = file_dict['FileStatus'][0]['blockSize']
length = file_dict['FileStatus'][0]['length']
length

174944099

In [5]:
#q9
block_distribution = {}
safe_list = []

for i in range(0, length, size):
    offset = str(i)
    url = "http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=" + offset + "&noredirect=true"
    response = requests.get(url)
    if response.status_code  == 403:
        block_distribution['lost'] = block_distribution.get('lost', 0) + 1
    else:
        safe_list.append(i)
        response = response.json()
        #print(response)
        loc = response['Location']
        #print(loc)
        datanode_id = loc.split("//")[1].split("/")[0].split(":")[0]
        block_distribution[datanode_id] = block_distribution.get(datanode_id, 0) + 1

block_distribution

{'43dc2ad30e77': 76, 'lost': 91}

In [6]:
hdfs = pa.fs.HadoopFileSystem("boss", 9000)
file_path = '/single.csv'  
f = hdfs.open_input_file(file_path)


2024-03-15 00:22:59,427 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
#q10
#create a safe list of offset
#interate through that list
#see if single family is existed in them
count = 0
for i in safe_list:
    #print(i)
    with hdfs.open_input_file(file_path) as file_obj:      
        content = f.read_at(size,i)
        count += content.count(b"Single Family")

count

203051